In [3]:
import numpy as np
import wave
from scipy.io import wavfile

# Implementación de Goertzel para una frecuencia específica
def goertzel(samples, target_freq, sample_rate):
    N = len(samples)
    k = int(0.5 + (N * target_freq) / sample_rate)  # Índice de la frecuencia objetivo
    omega = (2.0 * np.pi * k) / N
    coeff = 2.0 * np.cos(omega)
    
    s_prev = 0.0
    s_prev2 = 0.0
    
    # Algoritmo de recurrencia de Goertzel
    for sample in samples:
        s = sample + coeff * s_prev - s_prev2
        s_prev2 = s_prev
        s_prev = s
    
    # Calcular magnitud de la frecuencia objetivo
    power = s_prev2**2 + s_prev**2 - coeff * s_prev * s_prev2
    return power

# Cargar archivo de audio
def load_wav(filename):
    sample_rate, data = wavfile.read(filename)
    if len(data.shape) > 1:  # Si es estéreo, convertir a mono
        data = np.mean(data, axis=1)
    return sample_rate, data

# Detectar la frecuencia dominante en un rango cercano a la frecuencia objetivo
def detect_frequency(data, sample_rate, target_freq, freq_range=5):
    max_power = 0
    detected_freq = target_freq
    
    # Probar frecuencias en el rango [target_freq - freq_range, target_freq + freq_range]
    for f in np.arange(target_freq - freq_range, target_freq + freq_range, 0.1):
        power = goertzel(data, f, sample_rate)
        if power > max_power:
            max_power = power
            detected_freq = f
    
    return detected_freq

# Configuración
filename = 'D:/UTN/PES/repos/TPS finales/Afinador/Audios/ee2.wav'  # Nombre del archivo WAV
target_freq = 330.0               # Frecuencia de afinación para la primera cuerda
freq_range = 5.0                  # Rango en Hz alrededor de la frecuencia objetivo

# Cargar el archivo de audio
sample_rate, data = load_wav(filename)

# Detectar frecuencia dominante cercana a la frecuencia objetivo
detected_freq = detect_frequency(data, sample_rate, target_freq, freq_range)

# Comparar con la frecuencia objetivo
print(f"Frecuencia detectada: {detected_freq:.2f} Hz")
if abs(detected_freq - target_freq) <= 0.5:
    print("La cuerda está afinada.")
else:
    print(f"La cuerda está desfasada por {detected_freq - target_freq:.2f} Hz.")

# Puede que los armónicos del tono compliquen la solución
# Pensar en usar un filtro pasa banda para obtener solo la fundamental.

Frecuencia detectada: 329.10 Hz
La cuerda está desfasada por -0.90 Hz.
